# Calculate ILAMB Data
Creates annual, regridded data as well as month of max and climatology, from raw ILAMB data. Uses a config file (e.g. ILAMB_obs.cfg) to get information about where ILAMB data is located and what the files, models, and variable names are.

In [1]:
import os
import warnings
import xarray as xr

import fates_calibration_library.utils as utils
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.analysis_functions as analysis

# ignore warnings about latitude being outside of -90, 90
warnings.simplefilter("ignore", UserWarning)

In [2]:
# config file for ILAMB data
ilamb_cfg = '/glade/work/afoster/FATES_calibration/scripts/configs/ILAMB_obs.yaml'

# land area file
land_area_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")

In [3]:
# config dictionary for running functions
config_dict = {
    'top_dir': '/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA',  # location of raw ILAMB data
    'out_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs',  # where to put compiled ILAMB data
    'start_date': '2000-01-01',  # start date to filter to
    'end_date': '2014-12-31',  # end date to filter to
    'user': 'afoster@ucar.edu',  # user
    'clobber': False,  # overwrite files?
}

In [4]:
# read in the ILAMB data dictionary
ilamb_dict = utils.get_config_file(ilamb_cfg)

In [30]:
import pandas as pd

In [43]:
keys = list(ilamb_dict.keys())
df_list = []
for key in keys:
    model = key.split('_')[0]
    var = key.split('_')[1]
    df_list.append(pd.DataFrame({'model': [model],
                      'variable': [var]}))

In [44]:
df = pd.concat(df_list)

In [46]:
df[df.variable == 'GPP']

,model,variable
0,FLUXCOM,GPP
0,WECANN,GPP
0,GBAF,GPP


In [14]:
def adjust_lon(ds: xr.Dataset, lon_name: str, to_180=True) -> xr.Dataset:
    """Adjusts the longitude values of a dataset to be from 0-360 to -180 to 180

    Args:
        ds (xr.Dataset): Dataset
        lon_name (str): name of the longitude variable

    Returns:
        xr.Dataset: Dataset with the longitude values changes
    """

    if to_180:
        # adjust lon values to make sure they are within (-180, 180)
        ds["longitude_adjusted"] = xr.where(
            ds[lon_name] > 180, ds[lon_name] - 360, ds[lon_name]
        )
    else:
        ds["longitude_adjusted"] = ds[lon_name] % 360

    # reassign the new coords to as the main lon coords
    # and sort DataArray using new coordinate values
    ds = (
        ds.swap_dims({lon_name: "longitude_adjusted"})
        .sel(**{"longitude_adjusted": sorted(ds.longitude_adjusted)})
        .drop_vars(lon_name)
    )

    ds = ds.rename({"longitude_adjusted": lon_name})

    return ds

In [12]:
ds = xr.open_dataset('/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA/tas/CRU4.02/tas.nc')

In [15]:
ds_adj = adjust_lon(ds, 'lon', to_180=False)

In [17]:
ds_adj.to_netcdf('/glade/work/afoster/test_tas.nc')

In [5]:
# create a target grid to regrid to
target_grid = analysis.create_target_grid(land_area_ds, 'FSR')

In [6]:
# loop through dictionary and process the ILAMB observations
# script will skip files that already exist unless clobber: True
ilamb.get_all_ilamb_data(config_dict, ilamb_dict, target_grid)

File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_GPP.nc for FLUXCOM_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/WECANN_GPP.nc for WECANN_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/GBAF_GPP.nc for GBAF_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_NEE.nc for FLUXCOM_NEE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/AVHRR_LAI.nc for AVHRR_LAI exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/AVH15C1_LAI.nc for AVH15C1_LAI exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_LE.nc for FLUXCOM_LE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/CLASS_LE.nc for CLASS_LE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/WECANN_LE.nc for WECANN_LE exists, skippin

In [ ]:
# compile data into one dataset
compiled_ds = ilamb.compile_ilamb_datasets(config_dict['out_dir'], ilamb_dict,
                                          target_grid.area)
compiled_ds.to_netcdf(os.path.join('/glade/work/afoster/FATES_calibration',
                                  'observations/all_ILAMB_obs.nc'), mode="w")